# 2.6 MLflow in practice

# Scenario 2: A cross-functional team with one data scientist working on an ML model 

**MLflow setup**:
* **Tracking server**: yes, local server
* **Backend store**:   sqlite database
* **Artifact store**:  local filesystem

---

The experiments can be explored locally by accessing the local tracking server.  

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`

In [1]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/artifacts_local/1', creation_time=1737130366722, experiment_id='1', last_update_time=1737130366722, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/artifacts_local/0', creation_time=1737130271046, experiment_id='0', last_update_time=1737130271046, lifecycle_stage='active', name='Default', tags={}>]

## Logistic regression in Iris Dataset  

Run on 'my-experiment-1'

Metadata about experiments are stored in sqlite db.  
Artifacts are stored in artifacts_local directory.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/01/17 16:32:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/artifacts_local/1/a360558fde6f4051b7d71760c802f899/artifacts'
🏃 View run omniscient-ram-556 at: http://127.0.0.1:5000/#/experiments/1/runs/a360558fde6f4051b7d71760c802f899
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/artifacts_local/1', creation_time=1737130366722, experiment_id='1', last_update_time=1737130366722, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/artifacts_local/0', creation_time=1737130271046, experiment_id='0', last_update_time=1737130271046, lifecycle_stage='active', name='Default', tags={}>]

---

## Interact with model registry

In [5]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [6]:
from mlflow.exceptions import MlflowException

client.search_registered_models()

[<RegisteredModel: aliases={'champion': '1'}, creation_timestamp=1737131132684, description='', last_updated_timestamp=1737131132703, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1737131132703, current_stage='None', description='', last_updated_timestamp=1737131132703, name='iris-classifier', run_id='fb0eda1b758547a6bdd59dccfcd7ed57', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/artifacts_local/1/fb0eda1b758547a6bdd59dccfcd7ed57/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]

Since we use a sqlite db as backend server we can **register** a model

### Register the model

In [7]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id  # get info about first run of experiment with ID 1
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2025/01/17 16:32:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1737131526046, current_stage='None', description='', last_updated_timestamp=1737131526046, name='iris-classifier', run_id='a360558fde6f4051b7d71760c802f899', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/artifacts_local/1/a360558fde6f4051b7d71760c802f899/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>

In [8]:
client.search_registered_models()

[<RegisteredModel: aliases={'champion': '1'}, creation_timestamp=1737131132684, description='', last_updated_timestamp=1737131526046, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1737131526046, current_stage='None', description='', last_updated_timestamp=1737131526046, name='iris-classifier', run_id='a360558fde6f4051b7d71760c802f899', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/04_use_case_scenarios/artifacts_local/1/a360558fde6f4051b7d71760c802f899/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>], name='iris-classifier', tags={}>]

### Set model Alias

In [9]:
client.set_registered_model_alias(
    name='iris-classifier', 
    alias='champion', 
    version='1',
    )